In [2]:
# Cell 1: Import Libraries
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

# Explanation: Standard imports for PyTorch and data.

In [3]:
# Cell 2: Load and Prepare Data (Same as Before)
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
train_dataset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transform)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=64, shuffle=False)

# Explanation: Reuse MNIST data with normalization.

In [4]:
# Cell 3: Define Improved CNN Model
class ImprovedCNN(nn.Module):
    def __init__(self):
        super(ImprovedCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)  # First conv layer
        self.conv2 = nn.Conv2d(32, 64, 3, 1)  # Second conv layer for more features
        self.dropout = nn.Dropout(0.5)  # Dropout to prevent overfitting
        self.fc1 = nn.Linear(64 * 5 * 5, 128)  # Fully connected (adjusted for pooling)
        self.fc2 = nn.Linear(128, 10)  # Output for 10 digits

    def forward(self, x):
        x = nn.ReLU()(self.conv1(x))
        x = nn.MaxPool2d(2)(x)  # Pooling reduces size
        x = nn.ReLU()(self.conv2(x))
        x = nn.MaxPool2d(2)(x)
        x = x.view(x.size(0), -1)  # Flatten
        x = self.dropout(nn.ReLU()(self.fc1(x)))
        x = self.fc2(x)
        return x

model = ImprovedCNN()
print("Improved Model Structure:")
print(model)

# Explanation: Added layers, pooling (shrinks images), and dropout (randomly drops neurons during training for robustness).

Improved Model Structure:
ImprovedCNN(
  (conv1): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
  (dropout): Dropout(p=0.5, inplace=False)
  (fc1): Linear(in_features=1600, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=10, bias=True)
)


In [5]:
# Cell 4: Loss, Optimizer, and Training
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

epochs = 5  # Train longer for better results
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f'Epoch {epoch+1}, Loss: {running_loss/len(train_loader):.4f}')

# Explanation: Same training loop; model should learn better with improvements.

Epoch 1, Loss: 0.2365
Epoch 2, Loss: 0.0842
Epoch 3, Loss: 0.0639
Epoch 4, Loss: 0.0516
Epoch 5, Loss: 0.0443
